## 步驟 1: 環境設定與授權

請在此處選擇您的操作模式、程式碼分支和啟動模式，然後點擊「確認設定並掛載雲端硬碟」。
- **操作模式**: 
  - `暫存模式`: 所有資料（包括資料庫）將在 Colab 執行階段結束後消失。AI 分析功能可能受限（若未設定API金鑰）。
  - `持久模式`: 需要授權 Google Drive，應用程式資料將儲存在您的雲端硬碟中 (位於 `/content/drive/MyDrive/wolfAI_Data`)，並在不同執行階段間保持。
- **GitHub 分支**: 指定要從哪個程式碼分支部署。
- **啟動模式**: 
  - `一般模式`: 以標準方式啟動服務。
  - `除錯模式`: 啟動服務時將輸出更詳細的日誌，方便排查問題。

In [ ]:
# --- Python 模組導入 ---
import os
import shutil
import time
import requests
import subprocess
import pytz
from datetime import datetime
from IPython.display import display, HTML, clear_output, Markdown
import ipywidgets as widgets
from google.colab import output, drive # 明確導入 drive

# --- 全域狀態變數 (用於 Cell 間協調) ---
settings_confirmed_success = False
services_attempted_start = False # 追蹤步驟 2 的按鈕是否已被點擊
port_conflict_occurred_global = False # 儲存 start.sh 中是否發生了端口衝突
backend_actually_ready_global = False # 追蹤後端健康檢查是否通過
launch_mode_value = "normal" # 預設值，將會被更新
operation_mode_value = "transient" # 預設值，將會被更新

# --- 全域設定 ---
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1"
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)
TAIPEI_TZ = pytz.timezone('Asia/Taipei')
PERSISTENT_DATA_PATH = "/content/drive/MyDrive/wolfAI_Data" # 持久化資料路徑

# --- UI 元件定義 ---
style = {'description_width': 'initial'}

operation_mode_selection = widgets.Dropdown(
    options=[
        ("暫存模式 (資料與設定不跨執行階段保存)", "transient"),
        ("持久模式 (資料與設定保存於Google Drive)", "persistent")
    ],
    value="transient",
    description="1. 操作模式:",
    style=style
)

branch_selection = widgets.Text(
    value="main",
    description="2. GitHub 分支:",
    style=style
)

launch_mode_selection = widgets.RadioButtons(
    options=["一般模式 (normal)", "除錯模式 (debug)"],
    description="3. 啟動模式:",
    disabled=False,
    style=style
)

confirm_settings_button = widgets.Button(
    description="確認設定並掛載雲端硬碟 (若選擇)",
    disabled=False,
    button_style="info",
    tooltip="點擊此按鈕以確認以上設定並準備環境。",
    icon="check"
)

settings_output = widgets.Output()

# --- Helper 函數 ---
def get_taipei_time_str():
    return datetime.now(TAIPEI_TZ).strftime("%Y-%m-%d %H:%M:%S %Z")

# --- 按鈕點擊事件處理函數 ---
def on_confirm_settings_clicked(b):
    global settings_confirmed_success, launch_mode_value, operation_mode_value
    settings_confirmed_success = False # 每次點擊時重置
    with settings_output:
        clear_output(wait=True)
        confirm_settings_button.disabled = True
        operation_mode_selection.disabled = True
        branch_selection.disabled = True
        launch_mode_selection.disabled = True

        # 將選擇的值儲存到全域變數，供其他儲存格使用
        operation_mode_value = operation_mode_selection.value
        launch_mode_value = launch_mode_selection.value.split("("[1].replace(");", "")
        branch_name_value = branch_selection.value

        print(f"[{get_taipei_time_str()}] ✅ 設定已確認")
        print(f"   - 操作模式: {operation_mode_value} ({operation_mode_selection.options[0 if operation_mode_value == "transient" else 1][0]}) ")
        print(f"   - 啟動模式: {launch_mode_value}")
        print(f"   - 程式碼分支: {branch_name_value}")
        print("-" * 50)

        os.environ["OPERATION_MODE"] = operation_mode_value
        # 後端 config.py 將會使用這些環境變數
        os.environ["LAUNCH_MODE"] = launch_mode_value 

        print(f"
[{get_taipei_time_str()}] 正在處理操作模式設定...")
        if operation_mode_value == "persistent":
            print(f"[{get_taipei_time_str()}] 持久模式：嘗試掛載 Google Drive...")
            try:
                drive.mount("/content/drive", force_remount=True)
                print(f"[{get_taipei_time_str()}] ✅ 成功: Google Drive 已成功掛載於 /content/drive。")
                os.makedirs(PERSISTENT_DATA_PATH, exist_ok=True)
                print(f"[{get_taipei_time_str()}] 提示: 應用程式資料將儲存於: {PERSISTENT_DATA_PATH}")
                # 設定持久模式下的資料庫路徑
                os.environ["REPORTS_DB_PATH"] = os.path.join(PERSISTENT_DATA_PATH, "reports.sqlite")
                os.environ["PROMPTS_DB_PATH"] = os.path.join(PERSISTENT_DATA_PATH, "prompts.sqlite")
                print(f"[{get_taipei_time_str()}] 提示: 持久模式下資料庫路徑已設定。")
                print(f"[{get_taipei_time_str()}] 提示: 請確保您已在 Colab Secrets 中設定持久模式所需的其他金鑰 (例如 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT, WOLF_IN_FOLDER_ID 等)。")
                settings_confirmed_success = True
            except Exception as e:
                print(f"[{get_taipei_time_str()}] ❌ 錯誤: 掛載 Google Drive 失敗: {e}")
                settings_confirmed_success = False
        else:
            print(f"[{get_taipei_time_str()}] 暫存模式：跳過 Google Drive 掛載。資料將在執行階段結束後清除。")
            # 暫存模式下，資料庫路徑將使用專案內 data/ 資料夾下的預設值
            os.environ.pop("REPORTS_DB_PATH", None) # 若先前執行時設定過，則移除
            os.environ.pop("PROMPTS_DB_PATH", None) # 若先前執行時設定過，則移除
            if not os.getenv("COLAB_GOOGLE_API_KEY"):
                print(f"[{get_taipei_time_str()}] ⚠️ 警告: 未偵測到 Colab Secret 中的 'COLAB_GOOGLE_API_KEY'。AI 分析功能 (Gemini) 將受限。")
            settings_confirmed_success = True
        
        if settings_confirmed_success:
            print(f"
[{get_taipei_time_str()}] ✅ 環境設定完成。請執行下一個 Cell 來安裝依賴並啟動服務。")
        else:
            print(f"
[{get_taipei_time_str()}] ❌ 環境設定未成功。請修正錯誤後重試。")

        # 重新啟用此儲存格的 UI 元件
        confirm_settings_button.disabled = False
        operation_mode_selection.disabled = False
        branch_selection.disabled = False
        launch_mode_selection.disabled = False

# --- 綁定事件並顯示UI ---
confirm_settings_button.on_click(on_confirm_settings_clicked)
display(
    widgets.VBox([
        operation_mode_selection,
        branch_selection,
        launch_mode_selection,
        confirm_settings_button
    ]),
    settings_output
)


## 步驟 2: 安裝依賴與啟動服務

點擊下方按鈕開始下載專案程式碼、安裝所需依賴，並啟動後端與前端服務。
**注意**: 此步驟可能需要幾分鐘時間，特別是首次執行時。

In [ ]:
start_services_button = widgets.Button(
    description="開始安裝與啟動服務",
    disabled=False,
    button_style="primary",
    tooltip="點擊此按鈕開始下載、安裝並啟動應用程式服務。",
    icon="cogs"
)
services_output = widgets.Output()

 # 儲存格 1 中的輔助函數 (如果儲存格獨立執行或順序錯亂，為清晰起見重定義，但全域範圍應能獲取)
# def get_taipei_time_str(): ... 

def run_shell_command(command, cwd=None):
    """執行 shell 命令並即時串流輸出。"""
     # 此函數也在儲存格 1 中定義。如果儲存格 1 已執行，此處重定義是無害的。
     # 如果為測試目的獨立執行此儲存格，則需要此定義。
    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        shell=True, text=True, bufsize=1, universal_newlines=True, cwd=cwd
    )
    for line in iter(process.stdout.readline, ""):
        print(line, end="")
    process.stdout.close()
    return process.wait()

def on_start_services_clicked(b):
    global services_attempted_start, port_conflict_occurred_global, backend_actually_ready_global
     services_attempted_start = True # 標記此按鈕已被點擊
     port_conflict_occurred_global = False # 每次嘗試時重置
     backend_actually_ready_global = False # 每次嘗試時重置

    with services_output:
        clear_output(wait=True)
        start_services_button.disabled = True

        if not settings_confirmed_success:
            print(f"[{get_taipei_time_str()}] ❌ 錯誤：請先完成步驟 1 的「環境設定與授權」並確保其成功完成。")
            start_services_button.disabled = False
            return

     # 從全域範圍獲取必要的值 (在儲存格 1 中設定)
     # 如果儲存格 1 已執行，launch_mode_value, operation_mode_value 應該可用。
        current_launch_mode = launch_mode_value
     current_branch_name = branch_selection.value # 或者也從儲存格 1 全域儲存此值

        print(f"[{get_taipei_time_str()}] 步驟 2.1: 複製專案程式碼 (分支: {current_branch_name})...")
        if os.path.exists(FULL_PROJECT_PATH):
            print(f"[{get_taipei_time_str()}] 偵測到已存在的專案目錄，正在移除: {FULL_PROJECT_PATH}")
            shutil.rmtree(FULL_PROJECT_PATH)
        
        clone_command = f"git clone --depth 1 -b {current_branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}"
        if run_shell_command(clone_command) != 0:
            print(f"[{get_taipei_time_str()}] ❌ 嚴重錯誤: 專案複製失敗。請檢查分支名稱和網路連線。")
            start_services_button.disabled = False
            return
        print(f"[{get_taipei_time_str()}] ✅ 成功: 專案已成功複製到 {FULL_PROJECT_PATH}。")

        print(f"
[{get_taipei_time_str()}] 步驟 2.2: 執行啟動腳本 (scripts/start.sh) 以模式: {current_launch_mode}...")
        start_script_path = os.path.join(FULL_PROJECT_PATH, "scripts", "start.sh")
        if not os.path.exists(start_script_path):
            print(f"[{get_taipei_time_str()}] ❌ 嚴重錯誤: 找不到啟動腳本 {start_script_path}。")
            start_services_button.disabled = False
            return
     os.chmod(start_script_path, 0o755) # 確保腳本可執行
        
        process = subprocess.run(
            ["bash", start_script_path, f"--mode={current_launch_mode}"], 
            capture_output=True, text=True, cwd=FULL_PROJECT_PATH
        )

     # 打印來自 start.sh 的標準輸出和標準錯誤
        if process.stdout:
            print(f"[{get_taipei_time_str()}] --- 啟動腳本標準輸出 ---
{process.stdout}")
        if process.stderr:
     # start.sh 的標準錯誤可能包含來自 npm/next 的正常進度訊息或實際錯誤
            print(f"[{get_taipei_time_str()}] --- 啟動腳本錯誤輸出 (可能包含進度或錯誤) ---
{process.stderr}")

        if process.returncode == 10 or \n           "ERROR_PORT_IN_USE_請注意端口已被佔用" in process.stdout or \n           "ERROR_PORT_IN_USE_請注意端口已被佔用" in process.stderr:
     port_conflict_occurred_global = True # 設定全域標記
     # 錯誤訊息將在儲存格 3 中顯示
            print(f"[{get_taipei_time_str()}] ❌ 偵測到端口衝突。請參閱步驟 3 的健康檢查結果以獲取詳細說明。")
     # 即使有端口衝突，也允許繼續到儲存格 3 以顯示 HTML 錯誤
        elif process.returncode != 0:
            print(f"[{get_taipei_time_str()}] ❌ 警告: 啟動腳本執行遭遇非預期錯誤 (返回碼: {process.returncode})。健康檢查可能會失敗。")
        else:
            print(f"[{get_taipei_time_str()}] ✅ 啟動腳本已執行完畢。")
     # 注意：start.sh 成功執行不保證服務已完全就緒。
     # 儲存格 3 中的健康檢查將驗證這一點。

        print(f"
[{get_taipei_time_str()}] ✅ 服務啟動流程已執行。請執行下一個 Cell 進行健康檢查並獲取訪問連結。")
        start_services_button.disabled = False

start_services_button.on_click(on_start_services_clicked)
display(start_services_button, services_output)


## 步驟 3: 健康檢查與獲取訪問連結

點擊下方按鈕以檢查後端和前端服務的狀態。如果服務均已就緒，將會顯示前端的訪問連結。
**注意**: 健康檢查可能需要 1-2 分鐘，請耐心等待。

In [ ]:
health_check_button = widgets.Button(
    description="執行健康檢查並獲取連結",
    disabled=False,
    button_style="success",
    tooltip="點擊以檢查服務狀態並獲取前端訪問連結。",
    icon="medkit"
)
health_check_output = widgets.Output()

PORT_ERROR_HTML = """
<div style=\"border: 2px solid red; padding: 10px; background-color: #ffebee;\">
<h3>⚠️ 應用程式啟動失敗！</h3>
<p><b>原因：</b>網路端口 3000 已被占用，可能是上一次的執行未完全關閉或有其他程式正在使用此端口。</p>
<p><b>解決方案：</b>請點選上方選單的【執行階段】->【中斷並重新啟動執行階段】，然後重新執行所有步驟。</p>
</div>
"""

GENERAL_ERROR_HTML = """
<h2>❌ 應用程式未能成功啟動。</h2>
<p>請檢查 Cell 2 的日誌輸出以了解詳細錯誤。您可以嘗試重新啟動執行階段並再次執行所有步驟。</p>
<p>如果問題持續，請考慮使用「除錯模式」以獲取更詳細的日誌輸出，並檢查 Colab Secret 中的 API 金鑰設定是否正確。</p>
"""

def format_component_status_html(components):
    if not components or not isinstance(components, dict):
        return ""
    
    html_parts = ["<p><b>各核心組件健康狀態：</b></p><ul>"]
    status_colors = {
        "正常": "green",
        "警告": "orange",
        "錯誤": "red",
        "未知": "grey"
    }
    for name, details in components.items():
        status = details.get("status", "未知")
        message = details.get("message", "無詳細訊息")
        color = status_colors.get(status, "grey")
        html_parts.append(f"<li><b>{name}</b>: <span style='color:{color};'>{status}</span> - {message}</li>")
    html_parts.append("</ul>")
    return "".join(html_parts)

def on_health_check_clicked(b):
    global backend_actually_ready_global # 更新全域狀態
    backend_actually_ready_global = False
    detailed_components_status_html = '' # 初始化，以便在不同流程分支中都能參考

    with health_check_output:
        clear_output(wait=True)
        health_check_button.disabled = True

        # 新增：檢查持久模式下 Drive 掛載是否成功的提示 (基於 settings_confirmed_success)
        if operation_mode_value == 'persistent' and not settings_confirmed_success:
            display(HTML("<p style='color:red;'><b>⚠️ 持久模式提醒:</b> Google Drive 可能未成功掛載，這將影響資料的持久化儲存。請檢查步驟 1 的輸出日誌以確認掛載狀態。</p>"))
        # 新增：檢查暫存模式下 COLAB_GOOGLE_API_KEY 是否設定的提示
        if operation_mode_value == 'transient' and not os.getenv('COLAB_GOOGLE_API_KEY'):
            # 嘗試從 Cell 1 的 settings_output 內容中確認是否已提示過 (更複雜的檢查，暫時簡化)
            # 目前的 settings_confirmed_success 可能不足以完全代表此狀態，因為它在暫存模式下即使沒有 KEY 也為 True
            display(HTML("<p style='color:orange;'><b>⚠️ 暫存模式提醒:</b> 未偵測到 Colab Secret 中的 'COLAB_GOOGLE_API_KEY'。AI 分析功能 (Gemini) 將因此受限。如果需要完整 AI 功能，請設定此金鑰。</p>"))
        time.sleep(0.1) # 小延遲確保HTML渲染

        if not services_attempted_start:
            print(f"[{get_taipei_time_str()}] ⚠️ 提示：請先執行步驟 2 的「安裝依賴與啟動服務」。")
            health_check_button.disabled = False
            return

        if port_conflict_occurred_global: # 檢查儲存格 2 設定的全域標記
            display(HTML(PORT_ERROR_HTML))
            print(f"[{get_taipei_time_str()}] 由於偵測到端口衝突，健康檢查未執行。")
            health_check_button.disabled = False
            return
        
        print(f"[{get_taipei_time_str()}] --- 後端健康檢查開始 ---
")
        backend_health_url = "http://localhost:8000/api/health"
        max_wait_seconds_backend = 120 # 等待後端最長時間 (2 分鐘)
        poll_interval = 10 # 輪詢間隔 (秒)
        start_time_backend = time.time()
        backend_ready = False

        while time.time() - start_time_backend < max_wait_seconds_backend:
            print(f"[{get_taipei_time_str()}] 正在檢查後端服務狀態 ({backend_health_url})...")
            try:
                response = requests.get(backend_health_url, timeout=8)
                if response.status_code == 200:
                    health_data = response.json()
                    # 新增：解析並顯示更詳細的後端組件健康狀態
                    if 'components' in health_data and isinstance(health_data['components'], dict):
                        detailed_components_status_html = format_component_status_html(health_data['components'])
                        if detailed_components_status_html:
                            display(HTML(detailed_components_status_html))
                    print(f"[{get_taipei_time_str()}] ✅ 後端服務回應正常: {health_data.get('message', '狀態良好')}")
                    if health_data.get('status') == '正常' or health_data.get('status') == '警告':
                         backend_ready = True
                         backend_actually_ready_global = True
                         break
                    else:
                        print(f"[{get_taipei_time_str()}] ⚠️ 後端服務狀態為 '{health_data.get('status')}'，訊息: {health_data.get('message')}。繼續等待...")
                else:
                    print(f"[{get_taipei_time_str()}] ⏳ 後端健康檢查返回狀態碼 {response.status_code}。將在 {poll_interval} 秒後重試...")
            except requests.exceptions.ConnectionError:
                print(f"[{get_taipei_time_str()}] ⏳ 後端服務尚未就緒 (連線錯誤)。將在 {poll_interval} 秒後重試...")
            except requests.exceptions.Timeout:
                print(f"[{get_taipei_time_str()}] ⏳ 後端健康檢查請求超時。將在 {poll_interval} 秒後重試...")
            except Exception as e:
                print(f"[{get_taipei_time_str()}] ⏳ 健康檢查時發生未知錯誤: {e}。將在 {poll_interval} 秒後重試...")
            
            time.sleep(poll_interval)
        
        if not backend_ready:
            print(f"
[{get_taipei_time_str()}] ❌ 後端服務在 {max_wait_seconds_backend} 秒內未能成功啟動或回應不健康。")
            display(HTML(GENERAL_ERROR_HTML))
            if detailed_components_status_html: # 如果在失敗前有獲取到部分組件資訊
                display(HTML("<br/><b>嘗試獲取到的後端組件狀態：</b><br/>" + detailed_components_status_html))
            health_check_button.disabled = False
            return
        
        print(f"
[{get_taipei_time_str()}] --- 前端服務啟動檢查開始 ---
")
        frontend_ready = False
        frontend_url = ""
        try:
            frontend_url = output.eval_js(f"google.colab.kernel.proxyPort(3000, {{'cache_bust': True}})" )
            print(f"[{get_taipei_time_str()}] 偵測到前端代理 URL: {frontend_url}")
        except Exception as e:
            print(f"[{get_taipei_time_str()}] ⚠️ 無法自動獲取前端 Colab 代理 URL: {e}")
            display(HTML("<h2>⚠️ 前端檢查跳過</h2><p>無法獲取前端的 Colab 代理 URL。您可以嘗試手動在 Colab 的輸出上方尋找端口 3000 的連結。</p>"))
            if backend_actually_ready_global and detailed_components_status_html: # 即使前端失敗，也顯示後端組件狀態
                 display(HTML("<br/>" + detailed_components_status_html))
            health_check_button.disabled = False
            return

        max_wait_seconds_frontend = 60
        start_time_frontend = time.time()
        while time.time() - start_time_frontend < max_wait_seconds_frontend:
            print(f"[{get_taipei_time_str()}] 正在檢查前端服務狀態 ({frontend_url})...")
            try:
                response_frontend = requests.get(frontend_url, timeout=8, headers={'User-Agent': 'ColabHealthCheck/1.0'})
                if response_frontend.status_code == 200:
                    print(f"[{get_taipei_time_str()}] ✅ 前端服務已就緒 (狀態碼 {response_frontend.status_code})。")
                    frontend_ready = True
                    break
                else:
                    print(f"[{get_taipei_time_str()}] ⏳ 前端服務返回狀態碼 {response_frontend.status_code}。將在 {poll_interval} 秒後重試...")
            except requests.exceptions.RequestException as e_req:
                print(f"[{get_taipei_time_str()}] ⏳ 前端服務尚未就緒 (請求錯誤: {e_req})。將在 {poll_interval} 秒後重試...")
            time.sleep(poll_interval)

        border = "="*50
        print(f"
{border}")
        if backend_ready and frontend_ready:
            success_html = f"<h2>✅ 應用程式已成功啟動！</h2><p>🔗 <b>前端訪問網址:</b> <a href='{frontend_url}' target='_blank'>{frontend_url}</a></p><p>如果頁面無法載入，請稍等幾分鐘後重試，或檢查是否有防火牆/網路問題。</p>"
            if backend_actually_ready_global and detailed_components_status_html:
                success_html += detailed_components_status_html
            display(HTML(success_html))
        elif backend_ready and not frontend_ready:
            display(HTML(f"<h2>⚠️ 後端已就緒，但前端未能確認！</h2><p>後端服務似乎已啟動，但前端介面 ({frontend_url}) 未能在時限內回應。</p><p>您可以嘗試手動點擊以下連結訪問前端：<a href='{frontend_url}' target='_blank'>{frontend_url}</a></p><p>如果無法訪問，請檢查 Cell 2 的日誌輸出以尋找前端相關的錯誤訊息。</p>"))
            if backend_actually_ready_global and detailed_components_status_html:
                display(HTML("<br/>" + detailed_components_status_html))
        else: 
            display(HTML(GENERAL_ERROR_HTML))
            if detailed_components_status_html: # 如果在失敗前有獲取到部分組件資訊
                display(HTML("<br/><b>嘗試獲取到的後端組件狀態：</b><br/>" + detailed_components_status_html))
        print(f"{border}
")

        health_check_button.disabled = False

health_check_button.on_click(on_health_check_clicked)
display(health_check_button, health_check_output)
